### Notebook for the label transfer of Healthy-CTRL to rest of samples and conditions with `scANVI` and high `n_latent`

- **Developed by:** Carlos Talavera-López Ph.D
- **Institute of Computational Biology - Computational Health Department - Helmholtz Munich**
- **Created on**: 221129
- **Last modified**: 240507

### Import required modules

In [ ]:
import scvi
import torch
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import plotnine as p
from pywaffle import Waffle
import matplotlib.pyplot as plt
from scib_metrics.benchmark import Benchmarker

### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

In [ ]:
warnings.simplefilter(action = 'ignore')
scvi.settings.seed = 1712
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"}
%config InlineBackend.figure_format = 'retina'
torch.set_float32_matmul_precision('medium')

In [ ]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 3,
)

### Read in Healthy data

In [ ]:
scCOPD_All = sc.read_h5ad('../data/Marburg_cell_states_locked_ctl231212.raw.h5ad')
scCOPD_All

In [ ]:
scCOPD_All.obs['group'].value_counts()

In [ ]:
scCOPD_All.obs['cell_states'].value_counts()

### Split by conditions

In [ ]:
reference = scCOPD_All[scCOPD_All.obs['group'].isin(['healthy_ctrl', 'copd_iav'])]
reference.obs['seed_labels'] = reference.obs['cell_states'].copy()
reference

In [ ]:
reference.obs['seed_labels'].value_counts()

In [ ]:
query = scCOPD_All[~scCOPD_All.obs['group'].isin(['healthy_ctrl', 'copd_iav'])]
query.obs['seed_labels'] = 'Unknown'
query

In [ ]:
query.obs['seed_labels'].value_counts()

In [ ]:
adata = reference.concatenate(query, batch_key = 'batch-scANVI', batch_categories = ['reference', 'query'], join = 'inner')
adata

### Perform further QC-filtering from prior exploratory analyses

In [ ]:
sc.pl.scatter(adata, x = 'total_counts', y = 'pct_counts_mt', color = "group", frameon = False)

In [ ]:
adata

### Select HVGs

In [ ]:
adata_raw = adata.copy()
adata.layers['counts'] = adata.X.copy()

sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 8000,
    layer = "counts",
    batch_key = "donor",
    subset = True
)
adata

### Transfer of annotation with scANVI

In [ ]:
scvi.model.SCVI.setup_anndata(adata,
                              batch_key = "donor", 
                            categorical_covariate_keys = ["illumina_stimunr"], 
                            labels_key = "seed_labels", 
                            layer = 'counts')

In [ ]:
scvi_model = scvi.model.SCVI(adata, 
                             n_latent = 250, 
                             n_layers = 3, 
                             dispersion = 'gene-batch', 
                             gene_likelihood = 'nb')

In [ ]:
scvi_model.train(60, 
                 check_val_every_n_epoch = 1, 
                 enable_progress_bar = True, 
                 accelerator = 'mps',
                 devices = [0])

In [ ]:
adata.obsm["X_scVI"] = scvi_model.get_latent_representation(adata)

### Evaluate model performance using the Svensson method

In [ ]:
history_df = (
    scvi_model.history['elbo_train'].astype(float)
    .join(scvi_model.history['elbo_validation'].astype(float))
    .reset_index()
    .melt(id_vars = ['epoch'])
)

p.options.figure_size = 12, 6

p_ = (
    p.ggplot(p.aes(x = 'epoch', y = 'value', color = 'variable'), history_df.query('epoch > 0'))
    + p.geom_line()
    + p.geom_point()
    + p.scale_color_manual({'elbo_train': 'black', 'elbo_validation': 'red'})
    + p.theme_minimal()
)

p_.save('scVI_model.png', dpi = 300)

print(p_)

### Label transfer with `scANVI` 

In [ ]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(scvi_model, 'Unknown')

In [ ]:
scanvi_model.train(25, 
                   check_val_every_n_epoch = 1, 
                   enable_progress_bar = True, 
                   accelerator = 'mps',
                   devices = [0])

In [ ]:
adata.obs["C_scANVI"] = scanvi_model.predict(adata)

In [ ]:
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

### Explore model performance using the Svensson method

In [ ]:
history_df = (
    scanvi_model.history['elbo_train'].astype(float)
    .join(scanvi_model.history['elbo_validation'].astype(float))
    .reset_index()
    .melt(id_vars = ['epoch'])
)

p.options.figure_size = 12, 6

p_ = (
    p.ggplot(p.aes(x = 'epoch', y = 'value', color = 'variable'), history_df.query('epoch > 0'))
    + p.geom_line()
    + p.geom_point()
    + p.scale_color_manual({'elbo_train': 'black', 'elbo_validation': 'red'})
    + p.theme_minimal()
)

p_.save('scANVI_model.png', dpi = 300)

print(p_)

### Visualise corrected dataset

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scANVI", n_neighbors = 250, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.2, spread = 4, random_state = 1712)
sc.pl.umap(adata, frameon = False, color = ['group', 'disease', 'infection', 'C_scANVI', 'seed_labels', 'donor', 'bd_rhapsody', 'illumina_stimunr'], size = 0.8, legend_fontsize = 5, ncols = 4)

In [ ]:
sc.pl.umap(adata, 
           frameon = False, 
           color = ['SMK', 'n_genes', 'doublet_scores', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'n_counts', 'sample_group', 'IAV_score'], 
           size = 0.6, 
           legend_fontsize = 5, 
           ncols = 4, 
           cmap = 'plasma')

### Compute integration metrics

In [ ]:
bm = Benchmarker(
    adata,
    batch_key = ["illumina_stimunr", "donor", "sample_group"],
    label_key = "C_scANVI",
    embedding_obsm_keys = ["X_pca", "X_scVI", "X_scANVI"],
    n_jobs = -1,
)
bm.benchmark()

In [ ]:
bm.plot_results_table(min_max_scale = False)

### Export annotated sample object 

In [ ]:
adata.obs.index = pd.Index(['-'.join(idx.split('-')[:3]) for idx in adata.obs.index])
adata.obs.index

In [ ]:
adata_raw.obs.index = pd.Index(['-'.join(idx.split('-')[:3]) for idx in adata_raw.obs.index])
adata_raw.obs.index

In [ ]:
adata.obs_names

In [ ]:
adata.obs['C_scANVI'].cat.categories

In [ ]:
adata.obs['C_scANVI'].value_counts()

### Export annotated object with raw counts

In [ ]:
adata

In [ ]:
adata_raw

In [ ]:
adata_export = anndata.AnnData(X = adata_raw.X, obs = adata.obs, var = adata_raw.var)
adata_export.obs['cell_states'] = adata.obs['C_scANVI'].copy()
adata_export.obsm['X_scVI'] = adata.obsm['X_scVI'].copy()
adata_export.obsm['X_umap'] = adata.obsm['X_umap'].copy()
adata_export.obsm['X_scANVI'] = adata.obsm['X_scANVI'].copy()
adata_export

In [ ]:
adata_export.write('../data/Marburg_cell_states_scANVI_ctl240504.raw.h5ad')